In [1]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset_23' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['chokepoint'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [2]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

2023-06-24 23:14:15.482840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:


import pandas as pd
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage, Image, QualityModel

session = database.session


# generamos un datafame con la información que necesitamos para los registros sin quality
# hacemos una query para actualizar los registros que no tienen quality


df_a = pd.read_sql_query(session.query(QualityImage,FaceImage.croppedImage_id)
    .join(FaceImage)
    .filter(QualityImage.quality != None,FaceImage.embeddingModel_id == 3).statement, con=session.bind)


sql_b = (session.query(QualityImage, FaceImage)
    .join(FaceImage)
    .filter(QualityImage.quality == None).limit(2000000).all())
n1 = 4000
n = n1
for Q,F in tqdm(sql_b):
    #recorremos el resultado y actualizamos el objeto QualityImage


    qm = Q.qualityModel_id
    em = F.embeddingModel_id
    ci = F.croppedImage_id

    df_r = df_a.loc[(df_a['qualityModel_id'] == qm) & (df_a['croppedImage_id'] == ci) ]
    assert len(df_r) == 1
    Q.quality = float(df_r.quality)
    if not n:
        session.commit()
        n = n1
    else:
        n-=1
session.commit() 
session.close()

100%|██████████| 263471/263471 [17:03<00:00, 257.35it/s]


: 

In [ ]:
query = (session.query(EnfsiPair).limit(100).all())

for img in query:
    filter = ['image_quality']
    cat = img.get_category(im_category_list = filter,fi_cat_list=[],detector='mtcnn',embedding_model='Arcface')

    cat2 = img.get_category([],[],'mtcnn','Arcface')

